In [3]:
import nipype
import os,glob,sys,shutil
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from IPython.display import Image

In [4]:
subjRootDir = "/data/HCP_BedpostData/"

FULL_SUBJECT_LIST = [x for x in os.listdir(subjRootDir) if os.path.isdir( subjRootDir+x+'/T1w/Diffusion')]
print(len(FULL_SUBJECT_LIST),"Subjects are potentially available to be processed!")

1007 Subjects are potentially available to be processed!


In [ ]:
subject_id_infosource = pe.Node(util.IdentityInterface(fields=['subject_id']),
                                name='subject_id_infosource')
subject_id_infosource.iterables = ('subject_id', FULL_SUBJECT_LIST)

In [ ]:
datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
                                               outfields=['dwi', 'bvec', 'bval']),
                                                 name='datasource')
datasource.inputs.base_directory = subjRootDir 
datasource.inputs.sort_filelist = True
datasource.inputs.template = '%s/T1w/Diffusion/%s'
datasource.inputs.template_args = dict(dwi=[['subject_id', 'data.nii.gz']],
                                       bvecs=[['subject_id', 'bvecs']],
                                       bvals=[['subject_id', 'bvals']],
                                       nodif_brain_mask=[['subject_id','nodif_brain_mask.nii.gz']])
## Just mapped each subject to the corresponding bvec,bvals, brain mask and preprocessed DWI data
### Create the Node for DTIFIT
dtifit = pe.Node(interface=fsl.DTIFit(), name='dtifit')


gen_fa = pe.Workflow(name="gen_fa")
gen_fa.base_dir = '/data/NipypeScratch/'
gen_fa.connect(subject_id_infosource, 'subject_id', datasource, 'subject_id')

gen_fa.connect(subject_id_infosource, 'subject_id', dtifit, 'base_name')
gen_fa.connect(datasource, 'bvecs', dtifit, 'bvecs')
gen_fa.connect(datasource, 'bvals', dtifit, 'bvals')
gen_fa.connect(datasource, 'nodif_brain_mask', dtifit, 'mask')
gen_fa.connect(datasource, 'dwi', dtifit, 'dwi')

In [ ]:
datasink = pe.Node(interface=nio.DataSink(), name="datasink")
datasink.inputs.base_directory = os.path.join('/data/NipypeScratch/',
                                              'dtifit_results')
datasink.inputs.parameterization = False
gen_fa.connect(dtifit, 'FA', datasink, 'FA')
gen_fa.connect(dtifit, 'MD', datasink, 'MD')
gen_fa.connect(dtifit, 'V1', datasink, 'V1')
gen_fa.connect(dtifit, 'V2', datasink, 'V2')
gen_fa.connect(dtifit, 'V3', datasink, 'V3')
gen_fa.connect(dtifit, 'L1', datasink, 'L1')
gen_fa.connect(dtifit, 'L2', datasink, 'L2')
gen_fa.connect(dtifit, 'L3', datasink, 'L3')

In [ ]:
print(gen_fa.write_graph(graph2use='colored',simple_form=False))
Image('/data/NipypeScratch/gen_fa/graph.png')
#Image('/data/HCP_Data/NipypeScratch/gen_fa/graph.png')

In [ ]:
gen_fa.run(plugin='MultiProc', plugin_args={'n_procs' : 40})

In [ ]:
#!ls /data/HCP_Data/HCP_BedpostData/106016/T1w/Diffusion/

In [ ]:
### Copy files in the dtifit results directory to the canonical location if the file isnt there yet
from os.path import join as oj
import shutil
fileType = ["FA", "L1", "L2", "L3", "MD", "V1", "V2", "V3"]
dtifit_NP_Results = "/data/NipypeScratch/dtifit_results/"

HCP_procData = "/data/HCP_BedpostData/"

for ft in fileType:
    ## going to loop through each fileType directory and grab the files and then copy them to the HCP Data directory
    ## if they don't exist 
    fl = os.listdir( oj(dtifit_NP_Results,ft) )
    for f in fl:
        subjID = f.split('_')[0]
        
        targetFile = oj(HCP_procData,subjID,"T1w","Diffusion","data_%s.nii.gz"%ft)
        srcFile = oj(dtifit_NP_Results,ft,f)
        #print(f,subjID,srcFile,targetFile)
        
        if not os.path.isfile(targetFile):
            shutil.copy(srcFile,targetFile)

In [ ]:
NEEDS_BEDPOSTX_RUN = [x for x in os.listdir(subjRootDir) if ( os.path.isdir( subjRootDir+x+'/T1w/Diffusion') and not os.path.isdir(subjRootDir+x+'/T1w/Diffusion.bedpostX'))  ]
print(len(NEEDS_BEDPOSTX_RUN),"Subjects NEED BedpostX To be Run!")

In [7]:
### Also create the nodif_brain and nodif images for any file that has DTIFIT run

from nipype.caching import Memory
mem = Memory(base_dir='/data/NipypeScratch/')

fsl_ExtractROI = mem.cache(fsl.ExtractROI)
fsl_ImageMaths = mem.cache(fsl.ImageMaths)


for s in FULL_SUBJECT_LIST:
    dtiData = "/data/HCP_BedpostData/%s/T1w/Diffusion/data.nii.gz" % s
    nodif_brain_mask = "/data/HCP_BedpostData/%s/T1w/Diffusion/nodif_brain_mask.nii.gz" % s
    
    
    ### These are the files I am trying to generate...
    nodif_brain =  "/data/HCP_BedpostData/%s/T1w/Diffusion/nodif_brain.nii.gz" % s
    nodif =  "/data/HCP_BedpostData/%s/T1w/Diffusion/nodif.nii.gz" % s

    if not os.path.isfile(nodif):
        print(nodif)
        try:
            r = fsl_ExtractROI(in_file=dtiData, t_min=0,t_size=1)
            print(r.outputs.roi_file)
        except:
            print("Subject %s doesn't have all DTI data??" % s)
    
        ## Now copy the generated nodif file to the canonical location
        shutil.copy(r.outputs.roi_file,nodif)
            
            
    if not os.path.isfile(nodif_brain):
        try:
            rim = fsl_ImageMaths(in_file=r.outputs.roi_file, op_string="-mas %s" % nodif_brain_mask)
        except:
            print("Subject %s could not run masking of original file" % s)
        shutil.copy(rim.outputs.out_file,nodif_brain)        

## Apply Brain Extraction Mask to T1 Image--

####right now we are using the default from freesurfer, but it looks kind of crappy



In [ ]:
for s in FULL_SUBJECT_LIST:
    ### These are the files I am trying to generate...
    T1 =  "/data/HCP_BedpostData/%s/T1w/T1w_acpc_dc.nii.gz" % s
    T1_MASK_FS =  "/data/HCP_BedpostData/%s/T1w/brainmask_fs.nii.gz" % s
    T1_BRAIN_FS="/data/HCP_BedpostData/%s/T1w/T1w_acpc_dc_masked.nii.gz" %s
    
    if not os.path.isfile(T1_BRAIN_FS):
#    if os.path.isfile(T1) and os.path.isfile(T1_MASK_FS):
        if os.path.isfile(T1) and os.path.isfile(T1_MASK_FS):
            try:
                rim = fsl_ImageMaths(in_file=T1, op_string="-mas %s" % T1_MASK_FS)
                shutil.copy(rim.outputs.out_file,T1_BRAIN_FS)
            except:
                print("Subject %s could not run masking of original T1 file using freesurfer mask" % s)


/data/HCP_BedpostData/101006/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/101410/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/102513/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/106016/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/112516/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/122822/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/124624/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/128026/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/130821/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/132118/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/134324/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/138231/T1w/T1w_acpc_dc_masked.nii.gz Already had a mask
/data/HCP_BedpostData/144428/T1w/T1w_acpc_dc_masked.nii.gz Alrea

180921-12:44:35,323 workflow INFO:
	 [Node] Finished "63cc1a530383edecee70e568121048d8".
180921-12:44:35,480 workflow INFO:
	 [Node] Setting-up "a90c0c02940268d0967338a273fc97e1" in "/data/NipypeScratch/nipype_mem/nipype-interfaces-fsl-utils-ImageMaths/a90c0c02940268d0967338a273fc97e1".
180921-12:44:35,494 workflow INFO:
	 [Node] Running "a90c0c02940268d0967338a273fc97e1" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /data/HCP_BedpostData/115724/T1w/T1w_acpc_dc.nii.gz -mas /data/HCP_BedpostData/115724/T1w/brainmask_fs.nii.gz /data/NipypeScratch/nipype_mem/nipype-interfaces-fsl-utils-ImageMaths/a90c0c02940268d0967338a273fc97e1/T1w_acpc_dc_maths.nii.gz
180921-12:44:39,370 workflow INFO:
	 [Node] Finished "a90c0c02940268d0967338a273fc97e1".
180921-12:44:39,521 workflow INFO:
	 [Node] Setting-up "f940aa61d31217ca1b144ee310b9d87e" in "/data/NipypeScratch/nipype_mem/nipype-interfaces-fsl-utils-ImageMaths/f940aa61d31217ca1b144ee310b9d87e".
180921-1

180921-12:45:21,372 workflow INFO:
	 [Node] Finished "6ea6decfc52e53f27faf4882cf9b545f".
180921-12:45:21,522 workflow INFO:
	 [Node] Setting-up "ba2326b3ae49f3e657da50f82483ea23" in "/data/NipypeScratch/nipype_mem/nipype-interfaces-fsl-utils-ImageMaths/ba2326b3ae49f3e657da50f82483ea23".
180921-12:45:21,531 workflow INFO:
	 [Node] Running "ba2326b3ae49f3e657da50f82483ea23" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /data/HCP_BedpostData/453542/T1w/T1w_acpc_dc.nii.gz -mas /data/HCP_BedpostData/453542/T1w/brainmask_fs.nii.gz /data/NipypeScratch/nipype_mem/nipype-interfaces-fsl-utils-ImageMaths/ba2326b3ae49f3e657da50f82483ea23/T1w_acpc_dc_maths.nii.gz
180921-12:45:25,397 workflow INFO:
	 [Node] Finished "ba2326b3ae49f3e657da50f82483ea23".
180921-12:45:25,516 workflow INFO:
	 [Node] Setting-up "22d07e7859bbca484e48a94ee4a740cf" in "/data/NipypeScratch/nipype_mem/nipype-interfaces-fsl-utils-ImageMaths/22d07e7859bbca484e48a94ee4a740cf".
180921-1

180921-12:45:51,962 workflow INFO:
	 [Node] Running "969e0903de1808d75e12fe92678e20b0" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /data/HCP_BedpostData/106824/T1w/T1w_acpc_dc.nii.gz -mas /data/HCP_BedpostData/106824/T1w/brainmask_fs.nii.gz /data/NipypeScratch/nipype_mem/nipype-interfaces-fsl-utils-ImageMaths/969e0903de1808d75e12fe92678e20b0/T1w_acpc_dc_maths.nii.gz
180921-12:45:55,908 workflow INFO:
	 [Node] Finished "969e0903de1808d75e12fe92678e20b0".
180921-12:45:56,78 workflow INFO:
	 [Node] Setting-up "dec36917f1ad8957b8e199ff72cfc85c" in "/data/NipypeScratch/nipype_mem/nipype-interfaces-fsl-utils-ImageMaths/dec36917f1ad8957b8e199ff72cfc85c".
180921-12:45:56,88 workflow INFO:
	 [Node] Running "dec36917f1ad8957b8e199ff72cfc85c" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /data/HCP_BedpostData/111211/T1w/T1w_acpc_dc.nii.gz -mas /data/HCP_BedpostData/111211/T1w/brainmask_fs.nii.gz /data/Nipy

In [ ]:
" ".join(NEEDS_BEDPOSTX_RUN)
